In [15]:
import numpy as np
from scipy.sparse import csr_matrix


In [16]:

def lu(A):
    assert len(A.shape) == 2, "only 2d matrices are supported"
    assert A.shape[0] == A.shape[1], "only nxn matrices are supported"
    n = A.shape[0]

    L = np.zeros(A.shape)
    U = np.zeros(A.shape)

    for i in range(n):
        L[i, i] = 1
        for k in range(i, n):
            U[i, k] = A[i, k] - np.sum(L[i] * U[:, k])

        for k in range(i + 1, n):
            L[k, i] = (A[k, i] - np.sum(L[k] * U[:, i])) / U[i, i]

    return csr_matrix(L), csr_matrix(U)


In [ ]:
def inverse_matrix(L,U):
    return np.linalg.inv(L) * np.linalg.inv(U) #for now


In [17]:

matrix = csr_matrix([
    [1, 0, 3],
    [0, 3, 1],
    [0, 0, 6]
], shape=(3, 3))
L, U = lu(matrix)
print(L.toarray())
print(U.toarray())



[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[[1. 0. 3.]
 [0. 3. 1.]
 [0. 0. 6.]]
